In [4]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np

#Constants we are going to use
total_weeks = 8
hours_in_week = 24 * 7
hours_in_day = 24
delivery_interval = 7 #days
Dt = 6 #hours

#Model sets
tasks = ['T11', 'T12', 'T21', 'T22', 'T31', 'T32', 'T41', 'T42', 'T43', 'T51', 'T52']
units = ['U1', 'U2', 'U3', 'U4', 'U5']
time_points = [i for i in range(int(total_weeks * hours_in_week / Dt) + 1)]
time_periods = [i for i in range(1, int(total_weeks * hours_in_week / Dt) + 1)]
materials = ['RM1', 'RM2', 'INT1', 'INT2', 'INT3', 'INT4', 'A', 'B1', 'B2', 'C1', 'C2']

#Model parameters
capacities = [80, 80, 5, 5, 5, 5, 10, 10, 10, 10, 10] #materials
production_costs = [1.0, 1.0, 0.5, 0.5, 1.5, 1.5, 1.0, 0.5, 0.5, 1.0, 0.5] #tasks
storage_costs = [0, 0, 0, 0, 0, 0, 1.5, 1.4, 1.8, 1.2, 2.0] #materials
daily_production_rate = [2, 1, 1, 1, 2, 2, 2, 1, 1, 1, 1] #tasks
production_rate = [rate * Dt / hours_in_day for rate in daily_production_rate]
onhand_inventory_list = [ 80, 80, 0, 0, 0, 0, 0, 0, 0, 0, 0]

key_value_pairs_storage_cost = zip(materials, storage_costs)
key_value_pairs_production_cost = zip(tasks, production_costs)
key_value_pairs_capacity = zip(materials, capacities)
key_value_pairs_production_rate = zip(tasks, production_rate)
key_value_pairs_onhand_inventory = zip(materials, onhand_inventory_list)

material_storage_cost = dict(key_value_pairs_storage_cost)
task_production_cost = dict(key_value_pairs_production_cost)
capacity = dict(key_value_pairs_capacity)
rate = dict(key_value_pairs_production_rate)
onhand_inventory = dict(key_value_pairs_onhand_inventory)

unit_can_process_tasks = {
  'U1': ['T11', 'T12'],
  'U2': ['T21', 'T22'],
  'U3': ['T31', 'T32'],
  'U4': ['T41', 'T42', 'T43'],
  'U5': ['T51', 'T52']
}

material_produced_by_tasks = {
  'RM1': [],
  'RM2': [],
  'INT1': ['T11', 'T21'],
  'INT2': ['T12', 'T22'],
  'INT3': ['T31'],
  'INT4': ['T32'],
  'A': ['T41'],
  'B1': ['T42'],
  'B2': ['T51'],
  'C1': ['T43'],
  'C2': ['T52']
}

material_consumed_by_tasks = {
  'RM1': ['T11', 'T21'],
  'RM2': ['T12', 'T22'],
  'INT1': ['T31', 'T41'],
  'INT2': ['T32'],
  'INT3': ['T42', 'T51'],
  'INT4': ['T43', 'T52'],
  'A': [],
  'B1': [],
  'B2': [],
  'C1': [],
  'C2': []
}

net_delivery = {}
for k in materials:
  for n in time_points:
    net_delivery[(k,n)] = 0

fp_deliveries_string = "3.0 2.5 3.5 2.5 2.5\
                        3.5 3.0 3.0 2.0 3.5\
                        2.5 2.5 2.5 1.0 1.5\
                        3.0 2.0 3.0 2.0 3.0\
                        1.5 2.0 1.5 0.5 3.0\
                        2.5 3.0 3.0 2.5 2.0\
                        4.0 0.0 1.0 2.0 1.5\
                        3.0 2.5 2.5 2.0 1.5"

fp_deliveries = np.array([-float(x) for x in fp_deliveries_string.split()])
fp_deliveries = fp_deliveries.reshape(8, 5)

for i, k in enumerate(materials[6:]):
  for j, n in enumerate(range(int(delivery_interval * (24 / Dt)), time_points[-1] + 1, int(delivery_interval * (24 / Dt)))):
    net_delivery[(k,n)] = fp_deliveries[j][i]

print(net_delivery)
m = gp.Model("production-planning-simple")
#Define model variables
w = m.addVars(tasks, time_periods, vtype=GRB.BINARY, name="w")
w_s = m.addVars(units, time_periods, vtype=GRB.BINARY, name="w_s")

s = m.addVars(materials, time_points[1:], vtype=GRB.CONTINUOUS, lb=0, name="s") # MAYBE ITS NOT AN INTEGER
#Define objective function
obj = sum( material_storage_cost[k] * Dt / hours_in_day * s[k,n] for k in materials for n in time_points[1:]) + \
      sum(task_production_cost[i] * rate[i] * w[i,n] for n in time_periods for i in tasks)

m.setObjective(obj, GRB.MINIMIZE)
#Define constraints
capacity_constraint = m.addConstrs(s[k,n] <= capacity[k] for k in materials for n in time_points[1:])

assignment = m.addConstrs(sum(w[i,n] for i in unit_can_process_tasks[j]) + w_s[j,n] == 1 \
  for j in units for n in time_periods)

initial_balance = m.addConstrs(s[k,1] == onhand_inventory[k] + net_delivery[k,1] \
  + sum(rate[i] * w[i,1] for i in material_produced_by_tasks[k]) \
  - sum(rate[i] * w[i,1] for i in material_consumed_by_tasks[k]) \
                             for k in materials )
                             
balance = m.addConstrs(s[k,n] == s[k,n-1] + net_delivery[k,n] \
  + sum(rate[i] * w[i,n] for i in material_produced_by_tasks[k]) \
  - sum(rate[i] * w[i,n] for i in material_consumed_by_tasks[k]) \
  for k in materials for n in time_points[2:] )
            
m.write('test.lp' )
m.optimize()

{('RM1', 0): 0, ('RM1', 1): 0, ('RM1', 2): 0, ('RM1', 3): 0, ('RM1', 4): 0, ('RM1', 5): 0, ('RM1', 6): 0, ('RM1', 7): 0, ('RM1', 8): 0, ('RM1', 9): 0, ('RM1', 10): 0, ('RM1', 11): 0, ('RM1', 12): 0, ('RM1', 13): 0, ('RM1', 14): 0, ('RM1', 15): 0, ('RM1', 16): 0, ('RM1', 17): 0, ('RM1', 18): 0, ('RM1', 19): 0, ('RM1', 20): 0, ('RM1', 21): 0, ('RM1', 22): 0, ('RM1', 23): 0, ('RM1', 24): 0, ('RM1', 25): 0, ('RM1', 26): 0, ('RM1', 27): 0, ('RM1', 28): 0, ('RM1', 29): 0, ('RM1', 30): 0, ('RM1', 31): 0, ('RM1', 32): 0, ('RM1', 33): 0, ('RM1', 34): 0, ('RM1', 35): 0, ('RM1', 36): 0, ('RM1', 37): 0, ('RM1', 38): 0, ('RM1', 39): 0, ('RM1', 40): 0, ('RM1', 41): 0, ('RM1', 42): 0, ('RM1', 43): 0, ('RM1', 44): 0, ('RM1', 45): 0, ('RM1', 46): 0, ('RM1', 47): 0, ('RM1', 48): 0, ('RM1', 49): 0, ('RM1', 50): 0, ('RM1', 51): 0, ('RM1', 52): 0, ('RM1', 53): 0, ('RM1', 54): 0, ('RM1', 55): 0, ('RM1', 56): 0, ('RM1', 57): 0, ('RM1', 58): 0, ('RM1', 59): 0, ('RM1', 60): 0, ('RM1', 61): 0, ('RM1', 62): 0, (